In [0]:
%pip install imbalanced-learn

In [0]:
# Importe de bibliotecas
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from collections import Counter

In [0]:
# Carrega os dados
df_01 = pd.read_csv("data/credit_card_parte1.csv", sep=";")
df_02 = pd.read_csv("data/credit_card_parte2.csv", sep=";")

# Unifica os dois DataFrames
df_full = pd.concat([df_01, df_02], axis=0)
print(df_01.shape[0])
print(df_02.shape[0])
print(df_full.shape[0])

In [0]:
display(df_full)

In [0]:
# Convert columns V1 - V28 to float64, replacing comma with dot
for i in range(1, 29):
    col = 'V' + str(i)
    df_full[col] = df_full[col].str.replace(',', '.').astype('float64')

In [0]:
display(df_full)

In [0]:
# --- 2.2 Separação de Dados (Features e Target) ---
X = df_full.drop('Class', axis=1)
y = df_full['Class']

# --- 2.3 Separação Treino e Teste (Estratificada) ---
# O 'stratify=y' garante que o conjunto de teste mantenha a proporção real de fraude.
X_treino, X_teste, y_treino, y_teste = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)   

print(f"Distribuição original no Treino: {Counter(y_treino)}")

# --- 2.4 Tratamento do Desbalanceamento com SMOTE (Apenas no Treino) ---
# Instale o imbalanced-learn no seu cluster Databricks, se necessário: %pip install imbalanced-learn
smote = SMOTE(random_state=42)

# Aplica o Oversampling (cria amostras sintéticas da classe minoritária)
X_balanceado, y_balanceado = smote.fit_resample(X_treino, y_treino)

print(f"\nDistribuição balanceada no Treino (após SMOTE): {Counter(y_balanceado)}")


In [0]:
display(y)

In [0]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report

# --- 3.1 Treinamento do Modelo (Random Forest) ---
modelo_rf = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')

print("\nIniciando Treinamento do Random Forest...")
# Treina com os dados BALANCEADOS
modelo_rf.fit(X_balanceado, y_balanceado)
print("Treinamento concluído!")

# --- 3.2 Previsão no Conjunto de Teste (Realista) ---
# Prevemos usando os dados de teste originais (com desbalanceamento real)
y_pred = modelo_rf.predict(X_teste)

# --- 3.3 Avaliação do Desempenho ---
print("\n--- Matriz de Confusão (Dados de Teste Reais) ---")
# Essencial para ver Falsos Negativos (fraudes não detectadas)
print(confusion_matrix(y_teste, y_pred))

print("\n--- Relatório de Classificação ---")
# Focar em Recall (Classe 1)
print(classification_report(y_teste, y_pred))

# Interpretação Rápida da Classe 1:
# Recall: Quão bem o modelo CAPTUROU as fraudes (alta prioridade).
# Precision: Quão preciso o modelo foi em suas previsões de fraude (baixa prioridade, mas importante para evitar falsos positivos).
